#### Setup

In [9]:
import itertools
import random

import numpy as np

In [10]:
patterns = [np.matrix([[x], [y]]) for x, y in [(2,4), (1,0.5), (0.5,1.5), (0,0.5)]]
labels = [1, 1, -1, -1]

In [11]:
def get_shuffled(labels, patterns):
    zipped = list(zip(labels, patterns))
    random.shuffle(zipped)
    labels_shuf, patterns_shuf = list(zip(*zipped))
    return labels_shuf, patterns_shuf

#### a) How many iterations are required by the pattern-based perceptron learning rule in order to seperate classes A and B correctly if the weight vector w is initialized as (0, 1, −1) and step size η is set to 0.1?

In [19]:
from scipy.special import expit as sigmoid

class Perceptron:
    def __init__(self, initial_weights):
        self._initial_weights = initial_weights
        self.weights = initial_weights[:]
        
    def _add_bias(self, data):
        return np.vstack([[1], data])
    
    def reset(self):
        self.weights = self._initial_weights[:]
        
    def predict(self, data):
        if len(self.weights) - len(data) == 1:
            data = self._add_bias(data)
        return np.sign(sum(w*data[j] for j, w in enumerate(self.weights)))
    
    def fit(self, data, classes, learn_rate, max_iterations):
        raise NotImplemented()
    
    
class PatternPerceptron(Perceptron):
    def fit(self, data, classes, learn_rate=0.1, max_iterations=100):
        # Add bias to data
        data = [np.vstack([[1], d]) for d in data]
        def has_converged():
            return all(self.predict(x) == y
                                    for x, y in zip(data, classes))
        iterations = 0
        while iterations < max_iterations and not has_converged():
            iterations += 1
            # Error is here, we must not take *all* data into account, but merely a single one
            # that we pick sequentially
            # => Sample-based learning!
            currently_misclassified = [(x, y) for x, y in zip(data, classes)
                                       if self.predict(x) != y]
            for j in range(len(self.weights)):
                increase = learn_rate * sum(y*x[j] for x, y in currently_misclassified)
                self.weights[j] += increase
        if not has_converged():
            raise Exception("Did not converge after {} iterations"
                            .format(iterations))
        return iterations

In [20]:
p = PatternPerceptron(initial_weights=[0, 1, -1])
print("Answer: {} iterations are required.".format(
    p.fit(patterns, labels, learn_rate=0.1, max_iterations=2000)))

Answer: 1 iterations are required.


CORRECT

#### b) How many iterations are required if η = 0.25?    Is the order of the considered patterns relevant? If so, give an example, otherwise, prove it.

In [21]:
p.reset()
print("Answer: {} iterations are required with η = 0.25.".format(
    p.fit(patterns, labels, learn_rate=0.25, max_iterations=100)))

randomized_iters = []
for _ in range(100):
    p.reset()
    labels_shuf, patterns_shuf = get_shuffled(labels, patterns)
    randomized_iters.append(
        p.fit(patterns_shuf, labels_shuf, learn_rate=0.25,
              max_iterations=100))
print("Answer: With the patterns in shuffled order, we needed {} iterations, so "
      "the order does not seem to be relevant.".format(set(randomized_iters)))

Answer: 2 iterations are required with η = 0.25.
Answer: With the patterns in shuffled order, we needed {2} iterations, so the order does not seem to be relevant.


NOT CORRECT -> Should be 6 iterations
NOT CORRECT -> Does matter
??? How to prove that this must be the case ???

#### c) After how many iterations does the gradient-based learning rule terminate for both η? In this case: Is the order of the considered patterns relevant?

In [ ]:
class GradientPerceptron(Perceptron):
    def fit(self, data, classes, learn_rate=0.1, max_iterations=100):
        # Add bias to data
        data = [np.vstack([[1], d]) for d in data]
        has_converged = lambda: all(self.predict(x) == y
                                    for x, y in zip(data, classes))
        iterations = 0
        while iterations < max_iterations and not has_converged():
            iterations += 1
            # ERROR: According to the solution, we should go over *all* misclassified patterns??
            i = np.random.randint(len(data))
            # We only want to select misclassified patterns
            if classes[i] == self.predict(data[i]):
                continue
            for j in range(len(self.weights)):
                increase = learn_rate * classes[i] * data[i][j]
                self.weights[j] += increase
        if not has_converged():
            raise Exception("Did not converge after {} iterations"
                            .format(iterations))
        return iterations

In [ ]:
gp = GradientPerceptron(initial_weights=[0, 1, -1])
for rate in (0.1, 0.25):
    gp.reset()
    gp.fit(patterns, labels, learn_rate=rate, max_iterations=100)
    print("Answer: With a rate of {}, gradient-based learning terminates.".format(rate))

for rate in (0.1, 0.25):
    for _ in range(100):
        gp.reset()
        labels_shuf, patterns_shuf = get_shuffled(labels, patterns)
        gp.fit(patterns_shuf, labels_shuf, learn_rate=0.1,
              max_iterations=100)
    print("With a rate of {} and shuffled data and labels it seems to terminate as well.".format(rate))
